# Lesson 09 — Google ADK: A2A Server

This notebook shows how to build an **A2A server** using **Google Agent Development Kit** with your choice of model provider.

## What we build

```mermaid
flowchart TD
    Tool["FunctionTool\nget_policy_info"]
    LLM["LiteLlm\nGitHub Phi-4 or LocalFoundry"]
    Tool --> Agent["LlmAgent\nname · instruction · tools"]
    LLM  --> Agent
    Agent -->|"to_a2a() one-liner"| Server["Starlette ASGI app\nport 10091"]
    Server -->|"A2A JSON-RPC"| Client["Any A2A Client"]
```

**Same use case as Lesson 05–08:** insurance policy Q&A assistant.

### Model provider options

| Provider                    | Variable value   | What you need                                                                 |
| --------------------------- | ---------------- | ----------------------------------------------------------------------------- |
| **GitHub Models**           | `"github"`       | `GITHUB_TOKEN` in `.env` — [get one free](https://github.com/settings/tokens) |
| **AI Toolkit LocalFoundry** | `"localfoundry"` | VS Code AI Toolkit extension, model loaded on port 5272                       |

Set `PROVIDER` in cell 2 before running.

Port used: **10091** (notebooks) · Full lesson script uses 10002


In [ ]:
# ── Imports & environment setup ──────────────────────────────────
import os
import threading
import warnings

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv(raise_error_if_not_found=False))

# ── Model provider ────────────────────────────────────────────────
# "github"       — GitHub Models  (free, needs GITHUB_TOKEN in .env)
#                  https://github.com/settings/tokens
# "localfoundry" — AI Toolkit LocalFoundry  (local, no token needed)
#                  VS Code AI Toolkit → Models → Load a model → Run
PROVIDER = "github"  # ← change to "localfoundry" to use a local model

if PROVIDER == "github":
    os.environ["OPENAI_API_BASE"] = "https://models.inference.ai.azure.com"
    os.environ["OPENAI_API_KEY"] = os.environ.get("GITHUB_TOKEN", "")
    LITELLM_MODEL = "openai/Phi-4"
    if not os.environ["OPENAI_API_KEY"]:
        raise EnvironmentError("Set GITHUB_TOKEN in your .env or environment first")

elif PROVIDER == "localfoundry":
    os.environ["OPENAI_API_BASE"] = "http://localhost:5272/v1/"
    os.environ["OPENAI_API_KEY"] = "unused"  # LocalFoundry ignores the key
    LITELLM_MODEL = "openai/Phi-4-mini-instruct"  # ← change to your loaded model ID
    print("ℹ  AI Toolkit LocalFoundry — ensure a model is running on port 5272")

else:
    raise ValueError(f"Unknown PROVIDER: {PROVIDER!r}")

SERVER_PORT = 10091

print(f"✓ Provider  : {PROVIDER}")
print(f"✓ API base  : {os.environ['OPENAI_API_BASE']}")
print(f"✓ Model     : {LITELLM_MODEL}")
print(f"✓ Port      : {SERVER_PORT}")

In [ ]:
# ── Knowledge base (inline) ───────────────────────────────────────
POLICY = """
BRIGHT SHIELD INSURANCE — POLICY SUMMARY

Coverage Period:  January 1 – December 31 2025
Policy Number:    BS-2025-DEMO

DEDUCTIBLES
  Annual deductible (individual) : $500
  Annual deductible (family)     : $1,000
  Out-of-pocket maximum          : $3,500 / person

PREMIUMS
  Monthly premium (individual)   : $120
  Monthly premium (family)       : $340

COVERAGE
  Liability                      : $100,000 per incident
  Collision                      : Covered (ACV after deductible)
  Comprehensive                  : Covered (ACV after deductible)
  Rental car reimbursement       : Up to $35/day, 30-day maximum
  Emergency roadside assistance  : Included (unlimited calls)
  Medical payments               : $5,000 per person

EXCLUSIONS
  Intentional damage, racing, commercial use are NOT covered.

CLAIMS
  File within 30 days of incident.
  Contact: claims@brightshield.example  |  1-800-555-0199
"""

print(f"Policy document loaded ({len(POLICY)} chars)")

In [ ]:
# ── Build the Google ADK agent ────────────────────────────────────
#
# Pattern:
#   1. FunctionTool     — wraps a plain Python function as a tool
#   2. LiteLlm          — model provider (endpoint set by PROVIDER)
#   3. LlmAgent         — connects model + tools + instructions

from google.adk.agents import LlmAgent  # type: ignore[import-untyped]
from google.adk.models.lite_llm import LiteLlm  # type: ignore[import-untyped]
from google.adk.tools import FunctionTool  # type: ignore[import-untyped]


def get_policy_info(topic: str) -> str:
    """Return the insurance policy document.  topic is ignored."""
    return POLICY


model = LiteLlm(model=LITELLM_MODEL)

agent = LlmAgent(
    model=model,
    name="PolicyQAAgent",
    description="Insurance policy Q&A powered by Google ADK",
    instruction=(
        "You are a helpful insurance assistant for Bright Shield Insurance. "
        "Use the get_policy_info tool to look up policy details and answer "
        "questions accurately. Keep answers concise."
    ),
    tools=[FunctionTool(get_policy_info)],
)

print(f"✓ Agent created : {agent.name}")
print(f"  Provider      : {PROVIDER}  ({LITELLM_MODEL})")

In [ ]:
# ── Wrap with to_a2a() — ADK's one-liner A2A export ───────────────
#
# to_a2a() emits an EXPERIMENTAL UserWarning; suppress it so the
# notebook output stays clean.

from a2a.utils.a2a_tools import to_a2a  # type: ignore[import-untyped]

with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    app = to_a2a(agent, host="localhost", port=SERVER_PORT)

print("✓ A2A app built via to_a2a()")
print(f"  Agent card : http://localhost:{SERVER_PORT}/.well-known/agent-card.json")
print(f"  JSON-RPC   : POST http://localhost:{SERVER_PORT}/")

In [ ]:
# ── Start the server in a background thread ──────────────────────

import uvicorn


def _run_server():
    uvicorn.run(app, host="0.0.0.0", port=SERVER_PORT, log_level="warning")


server_thread = threading.Thread(target=_run_server, daemon=True)
server_thread.start()

import time

time.sleep(1.5)  # give uvicorn a moment to bind

import httpx

resp = httpx.get(
    f"http://localhost:{SERVER_PORT}/.well-known/agent-card.json", timeout=5
)
card = resp.json()
print(f"✓ Server live — status {resp.status_code}")
print(f"  Name        : {card.get('name')}")
print(f"  Description : {card.get('description')}")
print()
print("Now open client.ipynb to send A2A requests →")